In [1]:
import json
import csv
import itertools
import pandas as pd
import numpy as np

In [2]:
filename = "data.json"

In [3]:
df = pd.read_json(filename)

In [4]:
# distinct id of the users who has added item to the cart
user_distinct_ids = df[df.event == "Add to Cart Web"]

In [5]:
user_distinct_ids.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES05834,1999,http://flyrobe.com/women/lehengas?delivery_dat...,1503533058,NaN,NaN,NaN
1,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533208,NaN,NaN,NaN
2,Firefox,54.0,Mumbai,107,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533219,NaN,NaN,NaN
3,Mobile Safari,10.0,Mumbai,10,iPhone,15c82f54f9017e-0dc211ce35bbbc-64793c7f-3d10d-1...,4.0,Add to Cart Web,https://www.google.co.in/,FRES05560,3899,https://flyrobe.com/women/ethnic-gowns,1503533525,NaN,NaN,NaN
4,Chrome,59.0,Delhi,6,Android,15cb6e4205211-08b6c197de6bc7-3f112b20-38400-15...,4.0,Add to Cart Web,https://www.google.co.in/,FRES03765,1499,https://www.google.co.in/,1503536564,NaN,NaN,NaN


In [6]:
user_distinct_ids = user_distinct_ids['distinct_id'].tolist()

In [7]:
user_distinct_ids[0]

'1573d7c483eb3-06235568306ea58-4c594331-100200-1573d7c484011c'

In [8]:
user_events = df[df.distinct_id.isin(user_distinct_ids)]

In [9]:
user_events = user_events.sort_values(by=['time'])

In [10]:
user_events.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
1591546,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512008,NaN,NaN,NaN
80557,Chrome,56.0,Delhi,22,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,8.0,Add to Cart Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512045,NaN,NaN,NaN
420251,Mobile Safari,10.0,Mumbai,NaN,iPad,blissglobalkidzee@gmail.com,NaN,Details Web,$direct,FRES01492,149,NaN,1486512054,NaN,NaN,NaN
1239440,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512056,NaN,NaN,NaN
769396,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512058,NaN,NaN,NaN


In [11]:
# with open('user_events.json', 'w') as f:
#     f.write(df.to_json(orient='records', lines=True))

In [12]:
list1 = []; user_dict = {}
for index, each in user_events.iterrows():
    if each["distinct_id"] not in user_dict:
        user_dict[each["distinct_id"]] = [{each["event"]: [each["time"], each["parent_id"]]}]
    else:
        user_dict[each["distinct_id"]].append({each["event"]: [each["time"], each["parent_id"]]})

In [13]:
with open('user_event_data_21-02-2018.json', 'wt') as jsonfile:
    json.dump(user_dict, jsonfile)

In [14]:
# TODO: see for cases (need to check for the validity of the events) ['FRES06022'],
cycle = {}
cycles_for_correlation = []
for user, ev_tm_pi in user_dict.items():
    temp_cycle = []
    for events in ev_tm_pi:
        for key, value in events.items():
            temp_cycle.append(value[1])
            if key == "Add to Cart Web":
                try:
                    cycle[user].append(temp_cycle)
                except KeyError:
                    cycle[user] = [temp_cycle]
                cycles_for_correlation.append(temp_cycle)
                temp_cycle = []

In [15]:
cycles_for_correlation_unique = []

for element in cycles_for_correlation:
    temp_list = []
    for item in list(set(element[:-1])):
        temp_list.append(item)
    temp_list.append(element[-1])
    cycles_for_correlation_unique.append(temp_list)
        


cycles_for_correlation_unique

[['FRES02262', 'FRES02262'],
 ['FRES02262', 'FRES02262'],
 ['FRES03785',
  'FRES01733',
  'FRES01737',
  'FRES01498',
  'FRES01494',
  'FRES04581',
  'FRES01548',
  'FRES01492',
  'FRES03514',
  'FRES04214',
  'FRES01998',
  'FRES03514'],
 ['FRES04753',
  'FRES01683',
  'FRES03785',
  'FEXP00698',
  'FRES01733',
  'FRES05064',
  'FRES04581',
  'FRES04587',
  'FRES03771',
  'FEXP03725',
  'FRES01467',
  'FRES01498',
  'FRES01678',
  'FRES03860',
  'FRES01496',
  'FRES05269',
  'FRES01737',
  'FRES01510',
  'FRES04961',
  'FRES01775',
  'FRES04214',
  'FRES05553',
  'FRES01775'],
 ['FRES01668',
  'FRES01851',
  'FRES05269',
  'FRES01775',
  'FRES05264',
  'FRES05269'],
 ['FRES01733',
  'FRES05364',
  'FRES04215',
  'FRES05269',
  'FRES03514',
  'FRES05553',
  'FRES01733'],
 ['FRES04760', 'FRES01733', 'FRES04760'],
 ['FRES03785',
  'FEXP01131',
  'FEXP00698',
  'FRES01733',
  'FRES05532',
  'FRES05606',
  'FRES03783',
  'FRES01601',
  'FRES01736',
  'FRES05639',
  'FRES01686',
  'FRES0477

In [16]:
cycles_for_correlation_unique_updated = []
for cycle in cycles_for_correlation_unique:
    if len(cycle) != 1:
        cycles_for_correlation_unique_updated.append(cycle)

In [17]:
#cycles_for_correlation_unique_updated = cycles_for_correlation_unique_updated[:10]

In [18]:
count = 0
for element in cycles_for_correlation_unique_updated:
    if len(element) ==1:
        count += 1
count

0

In [19]:
brand_mapping = {}
with open('brand_mapping.json', 'r') as f:
     brand_mapping = json.load(f)
        
color_mapping = {}
with open('color_mapping.json', 'r') as f:
     color_mapping = json.load(f)

inventory_category_mapping = {}
with open('inventory_category_mapping.json', 'r') as f:
     inventory_category_mapping = json.load(f)

In [20]:
look_up_men = []
look_up_women = []
with open('parent_item_data_female_26-02-2018.json', 'r') as f:
     look_up_women = json.load(f)
        
with open('parent_item_data_male_26-02-2018.json', 'r') as f:
     look_up_men = json.load(f)

unique_inven_cat_men = []
unique_brand_ids_men = []
unique_color_men = []
for element in look_up_men.keys():
    if look_up_men[element]["inventory_category"] not in unique_inven_cat_men:
        if look_up_men[element]["inventory_category"] != None:
            unique_inven_cat_men.append(look_up_men[element]["inventory_category"])
        
    if look_up_men[element]["brand_id"] not in unique_brand_ids_men:
        if look_up_men[element]["brand_id"] != None:
            unique_brand_ids_men.append(look_up_men[element]["brand_id"])
            
    if look_up_men[element]["color"] not in unique_color_men:
        if look_up_men[element]["color"] != None:
            unique_color_men.append(look_up_men[element]["color"])
        
        
unique_inven_cat_women = []
unique_brand_ids_women = []
unique_color_women = []
for element in look_up_women.keys():
    if look_up_women[element]["inventory_category"] not in unique_inven_cat_women:
        if look_up_women[element]["inventory_category"] != None:
            unique_inven_cat_women.append(look_up_women[element]["inventory_category"])
        
    if look_up_women[element]["brand_id"] not in unique_brand_ids_women:
        if look_up_women[element]["brand_id"]  != None:
            unique_brand_ids_women.append(look_up_women[element]["brand_id"])
    
    if look_up_women[element]["color"] not in unique_color_women:
        if look_up_women[element]["color"]  != None:
            unique_color_women.append(look_up_women[element]["color"])



            
# later update this lookup
# fetch rentals from rds 
#unique_inven_cat_women
#look_up_men

In [21]:
# get all unique inventory_categories and brand_ids
unique_inven_cat_men = list(set(unique_inven_cat_men))
unique_brand_ids_men = list(set(unique_brand_ids_men))
unique_color_men = list(set(unique_color_men))
inventory_cat_correlation_matrix_men = np.identity(len(unique_inven_cat_men))
brand_correlation_matrix_men = np.identity(len(unique_brand_ids_men))
color_correlation_matrix_men = np.identity(len(unique_color_men))

unique_inven_cat_women = list(set(unique_inven_cat_women))
unique_brand_ids_women = list(set(unique_brand_ids_women))
unique_color_women = list(set(unique_color_women))
inventory_cat_correlation_matrix_women = np.identity(len(unique_inven_cat_women))
brand_correlation_matrix_women = np.identity(len(unique_brand_ids_women))
color_correlation_matrix_men = np.identity(len(unique_color_women))

### Inventory Category Correlation ###

In [22]:
tot_occ_map_men = {}
tot_occ_map_women = {}

# TODO: make different cor-mat for male/female


inventory_category_cycle_men = []
inventory_category_cycle_women = []

for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_men[element]['inventory_category']:
                local_list.append(look_up_men[element]['inventory_category'])
        inventory_category_cycle_men.append(local_list)
    except KeyError:
        pass


for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_women[element]['inventory_category']:
                local_list.append(look_up_women[element]['inventory_category'])
        inventory_category_cycle_women.append(local_list)
    except KeyError:
        pass


for cycle in inventory_category_cycle_men:
    for element in cycle[:-1]:
        if element not in tot_occ_map_men.keys() and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_men[element] = 1
        elif element != None and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_men[element] += 1

for cycle in inventory_category_cycle_women:
    for element in cycle[:-1]:
        if element not in tot_occ_map_women.keys() and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_women[element] = 1
        elif element != None and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_women[element] += 1



# cl = []
# for item in cycles_for_correlation_unique_updated:
#     for element in item[:-1]:
#         if element in look_up_men.keys():
#             if (look_up_men[element]['inventory_category'] in tot_occ_map_men.keys()) and look_up_men[element]['inventory_category'] != None and look_up_men[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_men[look_up_men[element]['inventory_category']] += 1
#             elif look_up_men[element]['inventory_category'] != None and look_up_men[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_men[look_up_men[element]['inventory_category']] = 1
#         elif element in look_up_women.keys():
#             #print(element)
#             if (look_up_women[element]['inventory_category'] in tot_occ_map_women.keys()) and look_up_women[element]['inventory_category'] != None and look_up_women[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_women[look_up_women[element]['inventory_category']] += 1
#             elif look_up_women[element]['inventory_category'] != None and look_up_women[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_women[look_up_women[element]['inventory_category']] = 1
                

#tot_occ_map_women

In [23]:
#tot_occ_map_men

In [24]:
#cycles_for_correlation_unique_updated

In [25]:
corelation_dict_ic_men = {}
unique_inven_cat_men = tot_occ_map_men.keys()
for element in sorted(unique_inven_cat_men):
    corelation_dict_ic_men[element] = {}
    for inner_element in sorted(unique_inven_cat_men):
        #if inner_element != element:
        corelation_dict_ic_men[element][inner_element] = 0


unique_inven_cat_women = tot_occ_map_women.keys()
corelation_dict_ic_women = {}
for element in sorted(unique_inven_cat_women):
    corelation_dict_ic_women[element] = {}
    for inner_element in sorted(unique_inven_cat_women):
        #if inner_element != element:
        corelation_dict_ic_women[element][inner_element] = 0


        
# corelation_dict
    

In [26]:

# cycles_for_correlation_unique_cat_men = []
# test_blazzers = 0
# cycles_for_correlation_unique_cat_women = []
# for element in cycles_for_correlation_unique:
#     for item in element[:-1]:
#         try:
#             if look_up_men[item]['inventory_category']:
#                 cycles_for_correlation_unique_cat_men.append(look_up_men[item]['inventory_category'])
#         except KeyError:
#             try:
#                 if look_up_women[item]['inventory_category']:
#                     cycles_for_correlation_unique_cat_women.append(look_up_women[item]['inventory_category'])
#             except KeyError:
#                 pass

#     try:
#         if look_up_men[element[-1]]['inventory_category']:
#             cycles_for_correlation_unique_cat_men.append(look_up_men[element[-1]]['inventory_category'])
#     except KeyError:
#         try:
#             if look_up_women[element[-1]]['inventory_category']:
#                 cycles_for_correlation_unique_cat_women.append(look_up_women[element[-1]]['inventory_category'])
#         except KeyError:
#             pass
        
# cycles_for_correlation_unique_cat_men.count(28), test_blazzers
#look_up_women

In [27]:
# for item in cycles_for_correlation:
#     for keys, values in corelation_dict.items():
#         for val in values.keys():
            
#             if corelation_dict[keys][val] == look_up[item[-1]]['inventory_category']:
#                 pass
#             else:
#                 corelation_dict[keys][look_up[item[-1]]['inventory_category']] += 1
#             try:
#                 if look_up[item[-1]]:
#                     corelation_dict[look_up[item[-1]]['inventory_category']] += 1
#             except KeyError:
#                 try:
#                     corelation_dict[look_up[item[-1]]['inventory_category']] = 1
#                 except KeyError:
#                     pass
count = 0
found = 0



print("Men")

for cycle in inventory_category_cycle_men:
    for key, value in corelation_dict_ic_men.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_ic_men[key][inner_key] += 1

# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_men[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_men.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_men[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_men[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_men[key][inner_key] += 1
#     except KeyError:
#         pass
#         #print(cycle[-1])
    
print("Women")

for cycle in inventory_category_cycle_women:
    for key, value in corelation_dict_ic_women.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_ic_women[key][inner_key] += 1


# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_women[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_women.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_women[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_women[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_women[key][inner_key] += 1
#     except KeyError:
#         pass
        #print(cycle[-1])
                
# for key, value in corelation_dict_ic_men.items(): #[ A: [A:, B:, C:, D:]] [iB1, i2 i3...i10]
#     for inner_key in value.keys():
#         for item in cycles_for_correlation_unique_updated:
#             #if key != inner_key:
#             #if item[-1] in look_up_men:
#             try:
#                 if inner_key == look_up_men[item[-1]]['inventory_category']:
#                     if inner_key == 29: 
#                         print(item[-1])
#                     for element in item[:-1]:
#                         if key == look_up_men[element]['inventory_category']:
#                             #print(key, element, inner_key)
#                             corelation_dict_ic_men[key][inner_key] += 1
#             except KeyError:
#                 pass
            
# for key, value in corelation_dict_ic_women.items(): #[ A: [A:, B:, C:, D:]] [iB1, i2 i3...i10]
#     for inner_key in value.keys():
#         for item in cycles_for_correlation_unique_updated:
#             #if key != inner_key:
#             #if item[-1] in look_up_men:
#             try:
#                 if inner_key == look_up_women[item[-1]]['inventory_category']:
#                     #print(item[-1], key, inner_key)
#                     for element in item[:-1]:
#                         if key == look_up_women[element]['inventory_category']:
#                             #print(key, element, inner_key)
#                             corelation_dict_ic_women[key][inner_key] += 1
#             except KeyError:
#                 pass
                #print(item[-1], key, inner_key)
            
#             elif item[-1] in look_up_women:
#                 if inner_key == look_up_women[item[-1]]['inventory_category']:
#                     for element in item[:-1]:
#                         if key == look_up_women[element]['inventory_category']:
#                             corelation_dict_ic_women[key][inner_key] += 1
#             try:
#                 if inner_key == look_up[item[-1]]['inventory_category']:
#                     for element in item[:-1]:
#                         if key == look_up[element]['inventory_category']:
#                             corelation_dict[key][inner_key] += 1
#                 found += 1
#             except KeyError:
#                 count += 1

Men
Women


In [28]:
#count, found
#corelation_dict_ic_women

In [29]:
for key, values in corelation_dict_ic_women.items():
    for inner_key in values.keys():
        try:
            #print(type(key), type(inner_key))
            #print(corelation_dict_ic_women[key][inner_key], tot_occ_map_women[inner_key], key, inner_key)
            corelation_dict_ic_women[key][inner_key] = float("%.4f" % (corelation_dict_ic_women[key][inner_key] / tot_occ_map_women[key]))
        except KeyError:
            pass
        
for key, values in corelation_dict_ic_men.items():
    for inner_key in values.keys():
        try:
            #print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], inner_key)
            corelation_dict_ic_men[key][inner_key] = float("%.4f" % (corelation_dict_ic_men[key][inner_key] / tot_occ_map_men[key]))
#             if corelation_dict_ic_men[key][inner_key] > 1:
#                 print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], key, inner_key)
        except KeyError:
            pass

In [30]:
#corelation_dict_ic_men, tot_occ_map_men

In [31]:
#corelation_dict_ic_women
for element, inner_key in corelation_dict_ic_men.items():
    print(inner_key, sum(list(inner_key.values())))
#     if round(sum(list(inner_key.values()))) != 1:
#         print(inner_key.values(), element, inner_key)

    


{23: 0.5668, 24: 0.0334, 28: 0.0603, 29: 0.1187, 30: 0.0818, 36: 0.096, 37: 0.0302, 38: 0.0126, 40: 0.0002} 1.0
{23: 0.0311, 24: 0.648, 28: 0.0221, 29: 0.0739, 30: 0.0357, 36: 0.1439, 37: 0.0401, 38: 0.0048, 40: 0.0005} 1.0001
{23: 0.0248, 24: 0.0064, 28: 0.8203, 29: 0.0445, 30: 0.065, 36: 0.0185, 37: 0.0041, 38: 0.0164, 40: 0.0001} 1.0001
{23: 0.0365, 24: 0.0222, 28: 0.0427, 29: 0.7691, 30: 0.0324, 36: 0.0602, 37: 0.0177, 38: 0.0189, 40: 0.0004} 1.0001000000000002
{23: 0.0421, 24: 0.0098, 28: 0.1005, 29: 0.0431, 30: 0.7539, 36: 0.0338, 37: 0.0074, 38: 0.0094, 40: 0.0} 1.0
{23: 0.0594, 24: 0.092, 28: 0.0348, 29: 0.0929, 30: 0.0436, 36: 0.6159, 37: 0.0464, 38: 0.0147, 40: 0.0004} 1.0001
{23: 0.0587, 24: 0.11, 28: 0.027, 29: 0.0947, 30: 0.0428, 36: 0.1512, 37: 0.5056, 38: 0.0095, 40: 0.0005} 0.9999999999999999
{23: 0.0384, 24: 0.0161, 28: 0.1007, 29: 0.123, 30: 0.0477, 36: 0.0661, 37: 0.0077, 38: 0.6003, 40: 0.0} 1.0
{23: 0.0128, 24: 0.0513, 28: 0.0256, 29: 0.1282, 30: 0.0, 36: 0.0385, 3

In [32]:
with open('men_ic_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_ic_men, jsonfile)
        
with open('women_ic_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_ic_women, jsonfile)

In [33]:
corelation_dict_ic_men_list = []
corelation_dict_ic_women_list = []
corelation_dict_ic_men_list = list(sorted(corelation_dict_ic_men.keys()))
corelation_dict_ic_women_list = list(sorted(corelation_dict_ic_women.keys()))

corelation_dict_ic_men_list_name = []
corelation_dict_ic_women_list_name = []

for element in corelation_dict_ic_men_list:
    corelation_dict_ic_men_list_name.append(inventory_category_mapping[str(element)])
    
for element in corelation_dict_ic_women_list:
    corelation_dict_ic_women_list_name.append(inventory_category_mapping[str(element)])


    

with open('men_inventory_category_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_ic_men_list = ["-"] + corelation_dict_ic_men_list_name
    wr.writerow(corelation_dict_ic_men_list)
    
    for key, val in sorted(corelation_dict_ic_men.items()):
        inner_list = [inventory_category_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

        
with open('women_inventory_category_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_ic_women_list = ["-"] + corelation_dict_ic_women_list_name
    wr.writerow(corelation_dict_ic_women_list)
    
    for key, val in sorted(corelation_dict_ic_women.items()):
        inner_list = [inventory_category_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)


In [34]:
for element, inner_key in corelation_dict_ic_men.items():
    print(sum(list(inner_key.values())))
    
for element, inner_key in corelation_dict_ic_women.items():
    print(sum(list(inner_key.values())))
    


1.0
1.0001
1.0001
1.0001000000000002
1.0
1.0001
0.9999999999999999
1.0
0.9999
0.9998
0.9999
1.0001999999999998
0.9998999999999998
1.0000999999999998
0.9999000000000002
1.0000000000000002
1.0000999999999998
1.0
0.9997999999999999
1.0002
1.0000999999999998
1.0001
0.9999000000000001
1.0000000000000002
1.0
0.9998999999999998
1.0000000000000002
1.0002
1.0001000000000002
1.0


### Brand Correlation ###

In [35]:
# brand correlation

tot_occ_map_men = {}
tot_occ_map_women = {}

# TODO: make different cor-mat for male/female


brand_cycle_men = []
brand_cycle_women = []

for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_men[element]['brand_id']:
                local_list.append(look_up_men[element]['brand_id'])
        brand_cycle_men.append(local_list)
    except KeyError:
        pass


for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_women[element]['brand_id']:
                local_list.append(look_up_women[element]['brand_id'])
        brand_cycle_women.append(local_list)
    except KeyError:
        pass


for cycle in brand_cycle_men:
    for element in cycle[:-1]:
        if element not in tot_occ_map_men.keys() and element != None:
            tot_occ_map_men[element] = 1
        elif element != None and element != None:
            tot_occ_map_men[element] += 1

for cycle in brand_cycle_women:
    for element in cycle[:-1]:
        if element not in tot_occ_map_women.keys() and element != None:
            tot_occ_map_women[element] = 1
        elif element != None and element != None:
            tot_occ_map_women[element] += 1


In [36]:
corelation_dict_brand_men = {}
unique_brand_ids_men = tot_occ_map_men.keys()
for element in sorted(unique_brand_ids_men):
    corelation_dict_brand_men[element] = {}
    for inner_element in sorted(unique_brand_ids_men):
        #if inner_element != element:
        corelation_dict_brand_men[element][inner_element] = 0


unique_brand_ids_women = tot_occ_map_women.keys()
corelation_dict_brand_women = {}
for element in sorted(unique_brand_ids_women):
    corelation_dict_brand_women[element] = {}
    for inner_element in sorted(unique_brand_ids_women):
        #if inner_element != element:
        corelation_dict_brand_women[element][inner_element] = 0




In [37]:
print("Men")

for cycle in brand_cycle_men:
    for key, value in corelation_dict_brand_men.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_brand_men[key][inner_key] += 1

# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_men[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_men.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_men[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_men[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_men[key][inner_key] += 1
#     except KeyError:
#         pass
#         #print(cycle[-1])
    
print("Women")

for cycle in brand_cycle_women:
    for key, value in corelation_dict_brand_women.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_brand_women[key][inner_key] += 1


Men
Women


In [38]:
for key, values in corelation_dict_brand_women.items():
    for inner_key in values.keys():
        try:
            #print(type(key), type(inner_key))
            #print(corelation_dict_ic_women[key][inner_key], tot_occ_map_women[inner_key], key, inner_key)
            corelation_dict_brand_women[key][inner_key] = float("%.4f" % (corelation_dict_brand_women[key][inner_key] / tot_occ_map_women[key]))
        except KeyError:
            pass
        
for key, values in corelation_dict_brand_men.items():
    for inner_key in values.keys():
        try:
            #print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], inner_key)
            corelation_dict_brand_men[key][inner_key] = float("%.4f" % (corelation_dict_brand_men[key][inner_key] / tot_occ_map_men[key]))
#             if corelation_dict_ic_men[key][inner_key] > 1:
#                 print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], key, inner_key)
        except KeyError:
            pass

In [39]:
corelation_dict_brand_men_list = []
corelation_dict_brand_women_list = []
corelation_dict_brand_men_list = list(sorted(corelation_dict_brand_men.keys()))
corelation_dict_brand_women_list = list(sorted(corelation_dict_brand_women.keys()))
    

    

    
    
    
corelation_dict_brand_men_list_name = []
corelation_dict_brand_women_list_name = []

for element in corelation_dict_brand_men_list:
    corelation_dict_brand_men_list_name.append(brand_mapping[str(element)])
    
for element in corelation_dict_brand_women_list:
    corelation_dict_brand_women_list_name.append(brand_mapping[str(element)])
    
    
    
with open('men_brand_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_brand_men_list = ["-"] + corelation_dict_brand_men_list_name
    wr.writerow(corelation_dict_brand_men_list)
    
    for key, val in sorted(corelation_dict_brand_men.items()):
        inner_list = [brand_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

        
with open('women_brand_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_brand_women_list = ["-"] + corelation_dict_brand_women_list_name
    wr.writerow(corelation_dict_brand_women_list)
    
    for key, val in sorted(corelation_dict_brand_women.items()):
        inner_list = [brand_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

In [40]:
for element, inner_key in corelation_dict_brand_men.items():
    #if sum(list(inner_key.values())) < 0.99:
    print(sum(list(inner_key.values())), element)
    
for element, inner_key in corelation_dict_brand_women.items():
    #if sum(list(inner_key.values())) < 0.99:
    print(sum(list(inner_key.values())), element)

1.0002 5
1.0002999999999993 6
1.0000000000000002 9
1.0000999999999998 49
0.9998999999999998 95
1.0001 166
1.0003 168
0.9999000000000002 169
1.0001999999999995 170
0.9993999999999998 171
0.9991999999999999 172
0.9999000000000001 173
0.9999999999999998 174
1.0001 175
0.9999999999999999 177
0.9997999999999997 179
1.0003000000000002 180
1.0001000000000002 197
1.0003000000000002 242
1.0002 243
0.9998 244
0.9998999999999999 257
1.0003999999999997 258
1.0 259
0.9998999999999998 260
1.0003999999999997 261
0.9999000000000001 278
0.9994999999999997 285
1.0002999999999997 290
1.0 295
0.9997000000000001 296
1.0 297
0.9999 298
1.0004000000000002 299
0.9995 300
0.9999000000000002 301
1.0 302
0.9996999999999999 303
0.9996999999999999 304
0.9998999999999998 309
0.9993 310
1.0000000000000002 311
1.0 312
0.9997999999999999 313
0.9997999999999998 317
0.9999 320
1.0002 321
1.0 322
1.0 331
1.0003 333
0.9999 358
0.9999 365
1.0 369
0.9999 374
0.9999 376
1.0 381
0.9993000000000009 1
0.9991999999999992 2
0.999

In [41]:
with open('men_brand_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_brand_men, jsonfile)
        
with open('women_brand_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_brand_women, jsonfile)

### Color Correlation ###

In [42]:
# color correlation

tot_occ_map_men = {}
tot_occ_map_women = {}

# TODO: make different cor-mat for male/female


color_cycle_men = []
color_cycle_women = []

for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_men[element]['color']:
                local_list.append(look_up_men[element]['color'])
        color_cycle_men.append(local_list)
    except KeyError:
        pass


for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_women[element]['color']:
                local_list.append(look_up_women[element]['color'])
        color_cycle_women.append(local_list)
    except KeyError:
        pass


for cycle in color_cycle_men:
    for element in cycle[:-1]:
        if element not in tot_occ_map_men.keys() and element != None:
            tot_occ_map_men[element] = 1
        elif element != None and element != None:
            tot_occ_map_men[element] += 1

for cycle in color_cycle_women:
    for element in cycle[:-1]:
        if element not in tot_occ_map_women.keys() and element != None:
            tot_occ_map_women[element] = 1
        elif element != None and element != None:
            tot_occ_map_women[element] += 1


In [43]:
corelation_dict_color_men = {}
unique_color_men = tot_occ_map_men.keys()
for element in sorted(unique_color_men):
    corelation_dict_color_men[element] = {}
    for inner_element in sorted(unique_color_men):
        #if inner_element != element:
        corelation_dict_color_men[element][inner_element] = 0


unique_color_women = tot_occ_map_women.keys()
corelation_dict_color_women = {}
for element in sorted(unique_color_women):
    corelation_dict_color_women[element] = {}
    for inner_element in sorted(unique_color_women):
        #if inner_element != element:
        corelation_dict_color_women[element][inner_element] = 0


In [44]:
color_cycle_women

[[25, 25, 25],
 [25, 25, 13, 25],
 [6,
  10,
  7,
  24,
  13,
  15,
  17,
  6,
  7,
  10,
  25,
  25,
  25,
  20,
  14,
  15,
  18,
  13,
  7,
  19,
  15,
  7,
  7,
  17,
  19,
  10,
  19,
  25,
  18,
  6,
  7,
  6,
  6,
  25,
  15,
  7,
  19,
  17,
  19,
  25,
  6,
  15,
  25,
  15,
  25,
  24,
  24,
  20,
  17,
  19,
  20],
 [18, 13, 25, 46, 6, 13],
 [19, 17, 6, 25, 25, 19],
 [7, 25, 46, 25, 7, 25],
 [7, 7],
 [7, 7, 7],
 [19, 13, 17, 7, 18, 2, 13],
 [14, 13, 13],
 [18, 18],
 [18, 13, 13],
 [18, 13, 18],
 [7,
  6,
  15,
  25,
  25,
  18,
  25,
  16,
  19,
  15,
  15,
  25,
  15,
  25,
  15,
  20,
  18,
  15,
  2,
  25,
  19,
  13,
  19,
  7,
  6,
  15,
  18,
  24,
  18,
  25,
  14,
  1,
  18,
  20,
  25,
  15,
  25,
  18],
 [15,
  25,
  16,
  20,
  7,
  19,
  7,
  18,
  25,
  19,
  46,
  46,
  24,
  14,
  15,
  6,
  24,
  18,
  17,
  18,
  10,
  15,
  7,
  15,
  18,
  15,
  15],
 [6, 6, 17, 7, 25, 16, 2, 19, 20, 19],
 [15, 16, 25, 14, 16],
 [6,
  1,
  14,
  16,
  17,
  16,
  19,
  19,

In [45]:
print("Men")

for cycle in color_cycle_men:
    for key, value in corelation_dict_color_men.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_color_men[key][inner_key] += 1

# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_men[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_men.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_men[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_men[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_men[key][inner_key] += 1
#     except KeyError:
#         pass
#         #print(cycle[-1])
    
print("Women")

for cycle in color_cycle_women:
    for key, value in corelation_dict_color_women.items():
        for inner_key, inner_val in value.items():
            try:
                if inner_key == cycle[-1]:
                    for element in cycle[:-1]:
                        if key == element:
                            #print(key, element, inner_key)
                            corelation_dict_color_women[key][inner_key] += 1
            except IndexError:
                print(cycle)



Men
Women
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

...

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [46]:
for key, values in corelation_dict_color_women.items():
    for inner_key in values.keys():
        try:
            #print(type(key), type(inner_key))
            #print(corelation_dict_ic_women[key][inner_key], tot_occ_map_women[inner_key], key, inner_key)
            corelation_dict_color_women[key][inner_key] = float("%.4f" % (corelation_dict_color_women[key][inner_key] / tot_occ_map_women[key]))
        except KeyError:
            pass
        
for key, values in corelation_dict_color_men.items():
    for inner_key in values.keys():
        try:
            #print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], inner_key)
            corelation_dict_color_men[key][inner_key] = float("%.4f" % (corelation_dict_color_men[key][inner_key] / tot_occ_map_men[key]))
#             if corelation_dict_ic_men[key][inner_key] > 1:
#                 print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], key, inner_key)
        except KeyError:
            pass

In [47]:
corelation_dict_color_men_list = []
corelation_dict_color_women_list = []
corelation_dict_color_men_list = list(sorted(corelation_dict_color_men.keys()))
corelation_dict_color_women_list = list(sorted(corelation_dict_color_women.keys()))

corelation_dict_color_men_list_name = []
corelation_dict_color_women_list_name = []

for element in corelation_dict_color_men_list:
    corelation_dict_color_men_list_name.append(color_mapping[str(element)])
    
for element in corelation_dict_color_women_list:
    corelation_dict_color_women_list_name.append(color_mapping[str(element)])
    
    

with open('men_color_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_color_men_list = ["-"] + corelation_dict_color_men_list_name
    wr.writerow(corelation_dict_color_men_list)
    
    for key, val in sorted(corelation_dict_color_men.items()):
        inner_list = [color_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

        
with open('women_color_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_color_women_list = ["-"] + corelation_dict_color_women_list_name
    wr.writerow(corelation_dict_color_women_list)
    
    for key, val in sorted(corelation_dict_color_women.items()):
        inner_list = [color_mapping[str(key)]]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

In [48]:
for element, inner_key in corelation_dict_color_men.items():
    #if sum(list(inner_key.values())) < 0.99:
    print(sum(list(inner_key.values())), element)
    
for element, inner_key in corelation_dict_color_women.items():
    #if sum(list(inner_key.values())) < 0.99:
    print(sum(list(inner_key.values())), element)

1.0000999999999998 1
1.0001 2
0.9999999999999997 6
0.9997000000000001 7
0.9999999999999998 9
1.0001 10
1.0002 11
1.0 12
1.0004 13
1.0001 14
0.9999000000000001 15
0.9998 16
0.9999000000000001 17
1.0 18
1.0 19
0.9998 20
1.0 21
0.9998999999999999 22
1.0 23
0.9996999999999999 24
1.0 25
1.0001 26
1.0002 31
1.0001 33
1.0 37
1.0001 39
0.9998999999999999 40
0.9998999999999998 44
1.0003999999999997 46
0.9998 49
1.0001 51
0.9999 56
0.9998 58
0.9997000000000001 62
1.0001 1
0.9998999999999999 2
0.9998999999999997 6
1.0002 7
1.0001999999999998 9
0.9999999999999996 10
1.0002 11
1.0002000000000004 12
0.9998999999999999 13
1.0004 14
0.9995999999999998 15
0.9997999999999998 16
1.0001999999999998 17
0.9999999999999998 18
1.0002999999999997 19
1.0003 20
0.9998 23
1.0005999999999997 24
0.9999 25
0.9998000000000001 26
1.0 28
0.9999000000000001 29
0.9999000000000001 31
0.9999 33
1.0 38
1.0 39
1.0 41
0.9999 42
0.9997999999999999 44
0.9997 46
0.9999 47
1.0 48
0.9999 49
0.9999 50
1.0 53
1.0 54
1.0001 55
1.0003

In [49]:
# import pandas
# import numpy

# dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32')]
# values = numpy.zeros(20, dtype=dtype)
# index = ['Row'+str(i) for i in range(1, len(values)+1)]

# df = pandas.DataFrame(values, index=index)
with open('men_color_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_color_men, jsonfile)
        
with open('women_color_correlation.json', 'wt') as jsonfile:
    json.dump(corelation_dict_color_women, jsonfile)

In [50]:
df.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES05834,1999,http://flyrobe.com/women/lehengas?delivery_dat...,1503533058,NaN,NaN,NaN
1,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533208,NaN,NaN,NaN
2,Firefox,54.0,Mumbai,107,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533219,NaN,NaN,NaN
3,Mobile Safari,10.0,Mumbai,10,iPhone,15c82f54f9017e-0dc211ce35bbbc-64793c7f-3d10d-1...,4.0,Add to Cart Web,https://www.google.co.in/,FRES05560,3899,https://flyrobe.com/women/ethnic-gowns,1503533525,NaN,NaN,NaN
4,Chrome,59.0,Delhi,6,Android,15cb6e4205211-08b6c197de6bc7-3f112b20-38400-15...,4.0,Add to Cart Web,https://www.google.co.in/,FRES03765,1499,https://www.google.co.in/,1503536564,NaN,NaN,NaN


In [51]:
float(3.0/37)

0.08108108108108109